In [2]:
#API et packages
import pandas as pd 
import requests
from faker import Faker 
import random 
from mistralai import Mistral 
import os 
import time 
import re 
from concurrent.futures import ThreadPoolExecutor, as_completed
#start_time = time.perf_counter()
#end_time = time.perf_counter()
#elapsed_time = (end_time - start_time)/60
#print(elapsed_time)
os.environ["MISTRAL_API_KEY"]="4n72jJx0kSjjxYwU7KP24xpEaRg2EAq3"
print("Clé API définie")

#initialisation 
api_key= os.getenv("MISTRAL_API_KEY") 
if not api_key: 
    raise ValueError("Clé API manquante. Ajoutez une clé valide pour Mistral.") 
print("Clé API récupérée") 
mistral = Mistral(api_key=api_key)
print("Objet Mistral initialisé") 

Clé API définie
Clé API récupérée
Objet Mistral initialisé


In [3]:
#test API
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "Parle moi de théorie des firmes",
        },
    ],
    max_tokens=1000
)


print(chat_response.choices[0].message.content)

La théorie des firmes est une branche de l'économie qui étudie les raisons pour lesquelles les entreprises existent, comment elles sont organisées, et comment elles prennent des décisions. Cette théorie s'intéresse à la manière dont les entreprises interagissent avec le marché, les consommateurs, les fournisseurs, et d'autres entreprises. Voici quelques concepts clés et théories majeures dans ce domaine :

### 1. **Théorie de la Firme de Ronald Coase**
Ronald Coase, dans son article "The Nature of the Firm" (1937), a proposé que les entreprises existent pour minimiser les coûts de transaction. Les coûts de transaction incluent les coûts de recherche d'informations, de négociation, de surveillance et d'application des contrats. En internalisant certaines transactions (c'est-à-dire en les réalisant au sein de l'entreprise plutôt que sur le marché), les entreprises peuvent réduire ces coûts.

### 2. **Théorie de l'Agence**
La théorie de l'agence examine les relations contractuelles où un 

In [4]:
#création du modèle de pricing 
#profil utilisateur= type de voiture (a,b,c,d,e,f,j,s,m)+ nombre de kilomètres par an + expérience de conduite(age-obtention du permis)+ âge +degré d'automatisation(par tranche de 10%)

#on génère un dataset pour faire notre modèle de pricing: 

#génération des types de voitures 
catégories=['A','B','C','D','E','F','J','M','S']
#A: mini-citadines, B: petites voitures, C:voitures compactes, D:grosses voitures, E:voitures de prestige, F:voitures de luxe, J:sport uility vehicule, M:grandes voitures familiales et S:voitures de sport
liste_catégorie=[]
while len(liste_catégorie)!=1000: 
    cat=random.randrange(0,9) 
    liste_catégorie.append(catégories[cat])
    
#génération des nombres de kilomètres par an 

#source:https://fr.statista.com/statistiques/899204/trajet-parcouru-en-voiture-sur-les-douze-derniers-mois-voiture-france/
#étant donné que les données sont sous la forme d'intervalle on va prendre en compte la médiane et les 3% qui restent sont répartis dans les autres catégories 
#31%=5000km; 55%=12500; 12%=37500; 2%=62500
import random

def generate_distances(total_count, percentages, distances):
    """
    Génère une liste de distances en fonction des pourcentages spécifiés.

    :param total_count: Nombre total de distances à générer.
    :param percentages: Liste des pourcentages correspondant à chaque distance.
    :param distances: Liste des distances correspondant à chaque pourcentage.
    :return: Liste de distances générées.
    """
    distance_list = []

    for percentage, distance in zip(percentages, distances):
        count = int(total_count * percentage / 100)
        distance_list.extend([distance] * count)
    while len(distance_list) < total_count:
        distance_list.append(distances[0]) 
    random.shuffle(distance_list)

    return distance_list

total_count = 1000  # Nombre total de distances à générer
percentages = [31, 55, 12, 2]
distances = [5000, 12500, 37500, 62500]

liste_distance = generate_distances(total_count, percentages, distances)

#génération de l'expérience de conduite 
liste_expérience=[]
liste_age=[]
while len(liste_expérience)!=1000:
    age=random.randrange(18,100)
    permis=random.randrange(18,100)
    if age>permis:
        expérience=age-permis 
        liste_expérience.append(expérience)    
        liste_age.append(age)

#génération du département
liste_département=[]
while len(liste_département)!=1000: 
    dép=random.randrange(1,97) 
    liste_département.append(dép)
    

#print des listes 
print(liste_catégorie)
print(liste_distance)
print(liste_age)
print(liste_expérience)
print(liste_département)

pricing_df = pd.DataFrame({'type de voiture':liste_catégorie,'expérience (en année)':liste_expérience,'age':liste_age,'distance parcouru par an (en kilomètres)':liste_distance, 'département':liste_département})
pricing_df.to_csv('pricing.csv')

['A', 'M', 'M', 'C', 'E', 'S', 'C', 'J', 'E', 'J', 'A', 'S', 'D', 'A', 'A', 'B', 'S', 'J', 'J', 'A', 'E', 'A', 'J', 'C', 'A', 'C', 'M', 'M', 'E', 'F', 'J', 'S', 'A', 'C', 'B', 'S', 'M', 'J', 'J', 'E', 'D', 'E', 'B', 'B', 'F', 'A', 'M', 'C', 'B', 'A', 'J', 'M', 'B', 'D', 'D', 'D', 'C', 'C', 'C', 'B', 'S', 'C', 'E', 'E', 'J', 'D', 'S', 'B', 'A', 'F', 'C', 'A', 'E', 'S', 'E', 'C', 'M', 'A', 'S', 'M', 'E', 'M', 'C', 'J', 'S', 'A', 'E', 'A', 'F', 'E', 'S', 'M', 'S', 'C', 'S', 'E', 'S', 'B', 'F', 'J', 'J', 'S', 'M', 'E', 'D', 'E', 'M', 'B', 'M', 'A', 'C', 'C', 'S', 'D', 'A', 'J', 'E', 'J', 'J', 'D', 'S', 'S', 'B', 'D', 'S', 'M', 'J', 'S', 'D', 'B', 'F', 'F', 'M', 'C', 'D', 'D', 'D', 'B', 'S', 'C', 'J', 'D', 'E', 'B', 'S', 'D', 'B', 'C', 'C', 'F', 'B', 'E', 'F', 'E', 'E', 'D', 'A', 'B', 'J', 'E', 'S', 'M', 'D', 'B', 'S', 'A', 'J', 'F', 'E', 'S', 'J', 'A', 'A', 'D', 'D', 'B', 'C', 'M', 'D', 'M', 'J', 'A', 'C', 'S', 'J', 'B', 'E', 'J', 'A', 'S', 'M', 'F', 'J', 'A', 'S', 'B', 'D', 'B', 'S', 'C',

In [6]:
automatisation=['0%','10%','20%','30%','40%','50%','60%','70%','80%','90%','100%']
liste_automatisation=[]
while len(liste_automatisation)!=1000: 
    cat=random.randrange(0,11) 
    liste_automatisation.append(automatisation[cat])
liste_automatisation

['20%',
 '10%',
 '50%',
 '60%',
 '20%',
 '80%',
 '50%',
 '30%',
 '100%',
 '80%',
 '100%',
 '20%',
 '40%',
 '70%',
 '90%',
 '40%',
 '100%',
 '50%',
 '0%',
 '60%',
 '80%',
 '90%',
 '90%',
 '10%',
 '60%',
 '90%',
 '10%',
 '70%',
 '70%',
 '80%',
 '70%',
 '50%',
 '30%',
 '40%',
 '90%',
 '20%',
 '60%',
 '40%',
 '60%',
 '0%',
 '0%',
 '100%',
 '70%',
 '30%',
 '40%',
 '50%',
 '100%',
 '100%',
 '30%',
 '50%',
 '60%',
 '50%',
 '30%',
 '30%',
 '20%',
 '50%',
 '90%',
 '90%',
 '20%',
 '50%',
 '20%',
 '30%',
 '30%',
 '40%',
 '20%',
 '20%',
 '60%',
 '80%',
 '80%',
 '80%',
 '70%',
 '0%',
 '40%',
 '90%',
 '60%',
 '20%',
 '10%',
 '30%',
 '70%',
 '50%',
 '80%',
 '90%',
 '30%',
 '0%',
 '30%',
 '60%',
 '20%',
 '70%',
 '50%',
 '50%',
 '80%',
 '100%',
 '70%',
 '0%',
 '80%',
 '10%',
 '90%',
 '70%',
 '40%',
 '30%',
 '90%',
 '60%',
 '20%',
 '80%',
 '40%',
 '0%',
 '20%',
 '30%',
 '10%',
 '0%',
 '100%',
 '100%',
 '30%',
 '40%',
 '10%',
 '20%',
 '30%',
 '0%',
 '100%',
 '40%',
 '50%',
 '60%',
 '30%',
 '40%',
 '0%',


In [7]:
pricing2_df = pd.DataFrame({
    'type de voiture': liste_catégorie,
    'expérience (en année)': liste_expérience,
    'age':liste_age,
    'distance parcourue par an (en kilomètres)': liste_distance,
    'département': liste_département,
    "degré d'automatisation de la conduite" : liste_automatisation
})
pricing2_df.to_csv('pricing2.csv', index=False)

In [8]:
#prix assurance2
answer = []

for i in range(len(pricing3_df)): 
    if i < 0 or i >= len(pricing3_df):
        print(f"Indice invalide : {i}")
        continue  

    prompt_content = f"Donne un prix précis (sous la forme suivante : x €) pour une assurance automobile tous risques en fonction des paramètres suivants sans donner d'explications : {pricing3_df.iloc[i].to_dict()}"
    try:
        chat_response = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt_content,
                },
            ],
            max_tokens=100,
            temperature=1
        )

        response_content = chat_response.choices[0].message.content.strip()
        print(response_content)
        time.sleep(2)  # Pause pour éviter un dépassement du quota API
        answer.append(response_content)

    except Exception as e:
        print(f"Erreur lors de la requête pour l'indice {i} : {e}")
        continue

#nettoyage des réponses 
answer2 = [x[-8:] for x in answer] 
answer3 = [x.replace(".", "").replace(" ", "").replace("€", "") for x in answer2] 

#dataframe
pricing3_df = pd.DataFrame({
    'type de voiture': liste_catégorie,
    'expérience (en année)': liste_expérience,
    'age':liste_age,
    'distance parcourue par an (en kilomètres)': liste_distance,
    'département': liste_département,
    'prix assurance': answer3,
    'automatisation': liste_automatisation
})
pricing3_df.to_csv('pricing3.csv', index=False)


950 €
485 €
500 €
1200 €
823 €
780 €
800 €
550 €
750 €
827 €
2500 €
1200 €
1250 €
850 €
2150 €
650 €
845 €
850 €
1200 €
2500 €
480 €
1200 €
850 €
800 €
370 €
650 €
1200 €
1500 €
950 €
1200 €
1200 €
1200 €
1350 €
1200 €
650 €
875 €.
500 €
850 €
857 €
580 €
650 €
780 €
980 €
1300 €
1350 €
900 €
680 €
1500 €
850 €
950 €
1200 €
628,70 €
950 €
840 €
1230 €
Impossible de fournir un prix précis sans données de tarification spécifiques. Le prix d'une assurance automobile tous risques dépend de nombreux facteurs, y compris des données spécifiques à l'assureur, des statistiques de sinistralité, et d'autres critères spécifiques à chaque compagnie d'assurance. Pour obtenir un devis précis, il est recomm
950 €
923 €
1200 €
1250 €
500 €
1150 €
450 €
650 €
1200 €
2500 €
850 €
Le prix précis pour une assurance automobile tous risques avec les paramètres fournis est de 1450 €.
956 €
750 €
1200 €
750 €
780 €
1200 €
720 €
1542 €
2150 €
1200 €
850 €
950 €
850 €
512 €
850 €
650 €
590 €
1200 €
550 €
750 €
8